In [55]:
from astropy.table import Table
from linedictionary import line_dict
from eDiskplot import linelabel_noqn
import pickle
import astropy.units as u
import astropy.constants as ac

c = ac.c.to(u.km / u.s).value


def dnu_to_dv(dnu, nu0):
    return dnu / nu0 * c


def get_spectroscopic_param_list(param_dict):
    return [
        str(param_dict["qn"]),
        str(param_dict["freq"]),
        "{:.3g}".format(param_dict["E_u"]),
        "{:.2f}".format(dnu_to_dv(param_dict["res"]*1e3, param_dict["freq"]*1e9)),
    ]

def get_beam_str(beam):
    bmajbmin = "{:.2f}$\\times${:.2f}".format(*beam[:2]).replace(".", "$\\farcs$")
    bpa = "({:.2g}$\\arcdeg$)".format(beam[2])
    return " ".join([bmajbmin, bpa])
    
get_beam_str((0.1, 0.1, 46))

'0$\\farcs$10$\\times$0$\\farcs$10 (46$\\arcdeg$)'

In [58]:
columns = [
    "Molecule",
    "Transition",
    "Rest Frequency",
    "$E_\\mathrm{up}$",
    "Spectral Resolution",
    "Channel Width",
    "Beam Size (PA)",
    "RMS",
]
units = ["", "", "[GHz]", "[K]", "[km s$^{-1}$]", "[km s$^{-1}$]", "", "[mJy beam$^{-1}$]"]
dtype = [str] * len(units)

with open("./data/L1489IRS_linecube_statistics.pkl", "rb") as f:
    stat = pickle.load(f)

table = Table(names=columns, dtype=dtype, units=units)
# table.add_row(units)

line_dict_keys = ["qn", "freq", "E_u", "res"]
molecules = ["13CO", "C18O", "SO"]

for line in molecules:
    # get spectroscopic params
    param_dict = line_dict[line]
    if "qn" in line_dict[line].keys():
        spectroparams = [linelabel_noqn[line]] + get_spectroscopic_param_list(param_dict)
    else:
        for i, t in enumerate(param_dict.keys()):
            param_dict = param_dict[t]
            if i == 0:
                spectroparams = [linelabel_noqn[line]] + get_spectroscopic_param_list(param_dict)
            else:
                spectroparams = [""] + get_spectroscopic_param_list(param_dict)

    # data params
    dataparams = [
        "{:.2f}".format(stat[line]["dchan"]),
        # "{:.2f}$\times${:.2f} ({:.2g})".format(*stat[line]["beam"]),
        get_beam_str(stat[line]["beam"]),
        "{:.2g}".format(stat[line]["rms"] * 1e3),
    ]

    table.add_row(spectroparams+dataparams)
table

Molecule,Transition,Rest Frequency,$E_\mathrm{up}$,Spectral Resolution,Channel Width,Beam Size (PA),RMS
,,[GHz],[K],[km s$^{-1}$],[km s$^{-1}$],,[mJy beam$^{-1}$]
str9,str12,str11,str4,str4,str4,str45,str3
$^{13}$CO,2--1,220.3986842,15.9,0.17,0.20,0$\farcs$13$\times$0$\farcs$11 (8.9$\arcdeg$),2.3
C$^{18}$O,2--1,219.5603541,15.8,0.17,0.20,0$\farcs$13$\times$0$\farcs$10 (13$\arcdeg$),1.7
SO,6$_5$--5$_4$,219.949442,35,0.17,0.20,0$\farcs$13$\times$0$\farcs$10 (15$\arcdeg$),2.1


In [59]:
table.write("./latex/cube_property_table.txt", format="aastex", overwrite=True)